## Building A Chatbot

In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.


In [59]:
import os
from dotenv import load_dotenv

load_dotenv()  ## aloading all the environment variable

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [60]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x120994750>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x12098b050>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [61]:
# Import the message types we need from langchain_core
from langchain_core.messages import HumanMessage, AIMessage

# Invoke the model with a conversation history
# This shows how we can pass multiple messages to maintain context
model.invoke(
    [
        HumanMessage(content="H i , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(
            content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"
        ),
        HumanMessage(content="Hey What's my name and what do I do?"),
    ]
)

AIMessage(content="According to our conversation, your name is Krish and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm interested to learn more! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148, 'completion_time': 0.090909091, 'prompt_time': 0.004367321, 'queue_time': 0.016535657, 'total_time': 0.095276412}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cca95fb4-eb49-4196-86b0-6a49f632ac43-0', usage_metadata={'input_tokens': 98, 'output_tokens': 50, 'total_tokens': 148})

### Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!


In [62]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}  ## dict to store chat history


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [63]:
config = {"configurable": {"session_id": "chat1"}}

In [64]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config,
)

In [65]:
response.content

"Hello Krish! It's nice to meet you.  \n\nThat's a very impressive title! As a Chief AI Engineer, I imagine you're involved in some exciting projects.  \n\nIs there anything I can help you with today? Perhaps you have a question about a particular AI technology, need help brainstorming ideas, or just want to discuss the latest advancements in the field?\n"

In [66]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Krish.  \n\nThat's what you told me at the beginning of our conversation! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 117, 'total_tokens': 142, 'completion_time': 0.045454545, 'prompt_time': 0.005333229, 'queue_time': 0.021759188999999998, 'total_time': 0.050787774}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-11be7757-1695-42a4-af70-4e1d87a691fa-0', usage_metadata={'input_tokens': 117, 'output_tokens': 25, 'total_tokens': 142})

In [67]:
## change the config-->session id
config1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")], config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn! 😊\n"

In [68]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")], config=config1
)
response.content

'Hi John!  Nice to meet you.  Is there anything I can help you with today? 😄  \n'

In [69]:
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")], config=config1
)
response.content

'Your name is John, you told me earlier! 😊  \n\nDo you have any other questions for me?  \n'

### Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.


In [70]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## MessagesPlaceholder is a placeholder for the messages in the prompt template
## in this case, we are using the variable name "messages"

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the question to the next of your ability",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [71]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [72]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")], config=config
)

response

AIMessage(content="Hello Krish, it's nice to meet you!\n\nI'm happy to help with any questions you have. Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 30, 'total_tokens': 63, 'completion_time': 0.06, 'prompt_time': 0.002286206, 'queue_time': 0.021277931, 'total_time': 0.062286206}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7e50a844-fede-4848-b4f6-74bcab7c58f5-0', usage_metadata={'input_tokens': 30, 'output_tokens': 33, 'total_tokens': 63})

In [73]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish.  😄 \n\nI remember that you told me!\n'

In [74]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.


In [75]:
with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="messages"
)

In [76]:
config = {"configurable": {"session_id": "chat4"}}
repsonse = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi,I am Krish")], "language": "Hindi"},
    config=config,
)
repsonse.content

'नमस्ते कृष्ण! \n\nआप कैसे हैं? 😊 \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। क्या मैं आपकी कोई मदद कर सकता हूँ? \n'

In [77]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [78]:
response.content

'आपका नाम कृष्ण है। 😊 \n'

### Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages


In [ ]:
# Import necessary message types and trim_messages utility
from langchain_core.messages import SystemMessage, trim_messages

# Create a trimmer that will limit conversation history
trimmer = trim_messages(
    max_tokens=45,  # Maximum tokens to keep in history
    strategy="last",  # Keep most recent messages
    token_counter=model,  # Use model to count tokens
    include_system=True,  # Always keep system message
    allow_partial=False,  # Don't split messages
    start_on="human",  # Start with a human message
)

# Example conversation history
messages = [
    SystemMessage(content="you're a good assistant"),  # System prompt
    HumanMessage(content="hi! I'm bob"),  # User messages
    AIMessage(content="hi!"),  # Assistant responses
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

# Trim the conversation history to fit within token limit
trimmer.invoke(messages)

/Users/shankii/Documents/AI/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [38]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do i like")],
        "language": "English",
    }
)
response.content

"As a helpful assistant, I don't have access to your personal information like your favorite ice cream flavor. \n\nWhat's your favorite ice cream flavor? 😊🍦\n"

In [39]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2".  \n\nDo you want to try another one? 😄 \n'

In [40]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "chat5"}}

In [41]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations or personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n"

In [42]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\nWhat's the problem? \n\n"